# PokeGAN
In our first GAN notebook, we looked at creating a StyleGAN to generate Pokemons. However, we saw that the generated images were not that satisfying, and the GAN was only able to capture the overall structures of Pokemons. 

In this one, we will use a pre-trained StyleGAN network and the original StyleGAN implementation from Nvidia to generate more satisfying Pokemons. And since we're using a pre-trained GAN, we don't even need to train the model and can go straight to generating images.

Since the original Nvidia codebase is written using `Tensorflow 1.x`, we also need to use that specific TF version.
> The Tensorflow version within Google Colab can always be changed using the magic command `%tensorflow_version 2.x`.

In [ ]:
%tensorflow_version 1.x

First, we need to clone the source code of Nvidia's StyleGAN Implementation, located on GitHub: https://github.com/NVlabs/stylegan.

Then we will need to download the weights of a pre-trained StyleGAN model. After correctly loading the model, it can be used to make predictions without any further training. Here we will use a model trained by Michael Friese ([tweet Michael Friese](https://twitter.com/MichaelFriese10/status/1127614400750346240)). The same model was also used to generate the Pokemon on this website: https://thesepokemondonotexist.com/.

In [ ]:
!git clone https://github.com/NVlabs/stylegan.git
!mv stylegan/* .
!rm -rf stylegan/
!wget https://s3.amazonaws.com/devopstar/resources/stylegan-pokemon/MichaelFriese10_pokemon.pkl

Import the needed packages. The `dnnlib` package is located in the same cloned repository from before.

Since we are using `TensorFlow 1.x`, a session needs to be initialized at the start. This will create the TensorFlow graph. Next, we will also need to load the StyleGAN configurations. These are then needed to instantiate the generator.

In [ ]:
import os
import pickle
import dnnlib.tflib as tflib
import numpy as np
import random
import matplotlib.pyplot as plt
from PIL import Image

# initialize the tensorflow session
tflib.init_tf()

# load StyleGAN configs
fmt = dict(func=tflib.convert_images_to_uint8, nchw_to_nhwc=True)
truncation = 0.5

Load the downloaded pre-trained pokemon model using `pickle`, a tool for serializing entire python objects. When we load the pickle file, we will get a generator, discriminator and style generator. For this exercise will only use the style generator.

In [ ]:
# specify the path of the downloaded model
model = "MichaelFriese10_pokemon.pkl"
# load the model using pickle
with open(model, 'rb') as f:
    _G, _D, Gs = pickle.load(f)

Use the loaded generator to create new Pokemons from a random latent vector. 

In [ ]:
# define our generation parameters
num_images = 10
scale = 15

# create our output folder for the generated pokemons
output_folder = "generated_pokemon"
output_file = "pokemon"
if not os.path.exists(output_folder):
  os.mkdir(output_folder)

# create random latent vectors
latents = np.random.RandomState(
    int(1000*random.random())).randn(num_images, *Gs.input_shapes[0][1:])
# pass the latent vectors to the generator, 
# this outputs our generated images
images = Gs.run(latents, None, truncation_psi=truncation,
                randomize_noise=True, output_transform=fmt)

# plot our grid of generated images
plt.figure(figsize=(scale*num_images, scale))
for i, img in enumerate(images):
  plt.subplot(1, num_images, i+1)
  plt.imshow(img)
  plt.axis('off')
  # save the pokemon
  image = Image.fromarray(img)
  image.save("{0}/{1}_{2}.png".format(output_folder, output_file, i+1))

As a final step, download our generated Pokemon.

In [ ]:
!zip -r ./generated_pokemon.zip ./generated_pokemon/
from google.colab import files
files.download("generated_pokemon.zip")
!rm -rf ./generated_pokemon